---
- SLNC의 2025.01월분 네이버 고객리뷰 데이터를, GPT API를 사용하여 감성분석 
- ref. https://colab.research.google.com/drive/1ord7ND2l0lvAhHyIfAMinxAOYBeUUlyj?usp=sharing#scrollTo=fcfcg517BGSa 
---

In [1]:
#!pip install --upgrade openai
#!pip install xlwings
import pandas as pd
import time
import openai
from openai import OpenAI, OpenAIError  # OpenAIError를 사용하여 모든 예외 처리
import xlwings as xw
from tqdm import tqdm

In [ ]:
# OpenAI 클라이언트 초기화 
client = OpenAI(api_key="xxxx") 

In [3]:
# 리뷰 데이터 로딩
'''
book = xw.Book('data/reviews_20250131.xlsx')
sheet = book.sheets[0]
df = sheet.used_range.options(pd.DataFrame, index=False).value  
'''
df = pd.read_excel('data/reviews_20250131.xlsx')
print(df.shape)
df.head(3).T

(1383, 11)


,0,1,2
brand,Chai797,Chai797,Chai797
type,Black,JUMBO,JUMBO
store,Chai797 Black 서래마을,Chai797 Jumbo 롯데대구점,Chai797 Jumbo 롯데대구점
date,2025-01-05 00:00:00,2025-01-02 00:00:00,2025-01-03 00:00:00
nickname,qkr****,mignon1018,yul****
content,굿,두번째 방문인데 완전 만족! 점보 탕수육도 짜장면도 전부 맛있어요! 다음에 또 와야...,우와~~ 칠리 새우 한마리가 입안 가득해요~ 탱글 촉촉 너무 맛났어요... 크림을...
revisit,1,1,1
reply_txt,NaN,고객님 안녕하십니까^^ 최고의 맛과 친절함을 보여드리기 위해 항상 노력하는 차이79...,고객님 안녕하십니까^^ 최고의 맛과 친절함을 보여드리기 위해 항상 노력하는 차이79...
reply_date,NaT,2025-01-13 00:00:00,2025-01-13 00:00:00
reply_due,NaN,11.0,10.0


In [4]:
def analyze_sentiment(review):
    """
    고객 리뷰를 분석하여 긍정/부정을 판단하는 함수
    """
    messages = [
        {"role": "system", "content": "너는 식당 매장에 대한 고객 리뷰에 담긴 고객 감정을 분석하고 탐지하는 AI 언어모델이야"},
        {"role": "user", "content": f"다음 고객 리뷰를 분석해 고객 감정이 긍정인지 부정인지 판단해 알려줘. "
                                    f"대답은 다른 추가적인 설명 없이 '긍정' 또는 '부정' 둘 중 하나의 단어로 대답해야 해: {review}"}]
    
    while True:
        try:
            completion = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages,
                max_tokens=10,
                temperature=0
            )
            return completion.choices[0].message.content.strip()
        except openai.OpenAIError as e:
            handle_api_error(e)


def analyze_procon(review):
    """
    고객 리뷰에서 긍정 단어와 부정 단어를 추출하는 함수
    """
    messages = [
        {"role": "system", "content": "너는 식당의 고객리뷰를 기반으로 식당과 관련된 긍정 단어와 부정 단어를 추출하는 AI 언어 모델이야."},
        {"role": "user", "content": f"다음 식당 리뷰에서 식당에서의 경험을 평가하는 긍정 단어와 부정 단어를 쉼표(,)로 구분하여 나열해줘."
                                       f"추가적인 설명 없이 긍정 단어들만, 그리고 부정 단어들만 나열해줘: {review}"}
    ]
    
    while True:
        try:
            completion = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages,
                max_tokens=50,
                temperature=0
            )
            response = completion.choices[0].message.content.strip()
            
            # '긍정 단어:' 또는 '부정 단어:' 제거 후 리스트 추출
            response = response.replace("긍정 단어:", "").replace("부정 단어:", "").strip()
            
            if "\n" in response:
                positive_words, negative_words = response.split("\n", 1)
            elif "," in response:
                positive_words, negative_words = response.split(",", 1)
            else:
                positive_words, negative_words = response, ""

            return positive_words.strip(), negative_words.strip()
        except openai.OpenAIError as e:
            handle_api_error(e)


def handle_api_error(e):
    error_message = str(e).lower()
    if "rate limit" in error_message:
        retry_time = 30
        print(f"💡 Rate limit exceeded. Retrying in {retry_time} seconds...")
    elif "service unavailable" in error_message:
        retry_time = 10
        print("💡 OpenAI Service is unavailable. Retrying in 10 seconds...")
    elif "timeout" in error_message or "connection error" in error_message:
        retry_time = 15
        print(f"💡 API Connection/Timeout error: {e}. Retrying in {retry_time} seconds...")
    else:
        print(f"OpenAI API 에러 발생: {e}")
        return None
    time.sleep(retry_time)

tqdm.pandas()
df["Sentiment"] = df["content"].progress_apply(analyze_sentiment)
df[["Positive_words", "Negative_words"]] = df["content"].progress_apply(lambda x: pd.Series(analyze_procon(x)))

# 엑셀 파일 저장
output_path = "output/sentiment_analysis.xlsx"
df.to_excel(output_path, index=False)
print(f"분석 결과가 저장되었습니다: {output_path}")

100%|██████████| 1383/1383 [25:11<00:00,  1.09s/it] 


분석 결과가 저장되었습니다: output/sentiment_analysis.xlsx


---
#### cf. 이하는 Only 긍정 or 부정만 구분하는 코드
---

In [ ]:
def analyze_review(review):
    """
    고객 리뷰를 분석하여 긍정/부정을 판단하는 함수
    """
    messages = [
        {"role": "system", "content": "너는 식당 매장에 대한 고객 리뷰에 담긴 고객 감정을 분석하고 탐지하는 AI 언어모델이야"},
        {"role": "user", "content": f"다음 고객 리뷰를 분석해 고객 감정이 긍정인지 부정인지 판단해 알려줘. "
                                    f"대답은 다른 추가적인 설명 없이 '긍정' 또는 '부정' 둘 중 하나의 단어로 대답해야 해: {review}"}
    ]

    while True:  # 실패 시 재시도 루프
        try:
            completion = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages,
                max_tokens=3,
                temperature=0
            )
            response = completion.choices[0].message.content.strip()  # 응답에서 공백 제거
            return response

        except OpenAIError as e:
            # OpenAIError는 여러 가지 에러를 포함할 수 있으므로 메시지를 확인하여 분기 처리
            error_message = str(e).lower()

            if "rate limit" in error_message:
                retry_time = 30  # 기본 재시도 시간 30초
                print(f"💡 Rate limit exceeded. Retrying in {retry_time} seconds...")
                time.sleep(retry_time)

            elif "service unavailable" in error_message:
                retry_time = 10  # 서버 오류 시 10초 후 재시도
                print("💡 OpenAI Service is unavailable. Retrying in 10 seconds...")
                time.sleep(retry_time)

            elif "timeout" in error_message or "connection error" in error_message:
                retry_time = 15  # 네트워크 오류 시 15초 후 재시도
                print(f"💡 API Connection/Timeout error: {e}. Retrying in {retry_time} seconds...")
                time.sleep(retry_time)

            else:
                print(f"OpenAI API 에러 발생: {e}")
                return None  # 심각한 에러 발생 시 None 반환

# 엑셀에서 불러온 데이터(df)를 사용하여 감정 분석 진행
sentiments = []

for review in tqdm(df["content"], desc="감정 분석 진행 중"):
    sentiment = analyze_review(review)
    sentiments.append(sentiment)

df["Sentiment"] = sentiments  # 감정 분석 결과 추가

# 엑셀 파일로 저장
output_path = "output/sentiment_analysis_simple.xlsx"
df.to_excel(output_path, index=False)
print(f"감정 분석 결과가 저장되었습니다: {output_path}") 